# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [59]:
import sqlite3
import pandas as pd

In [60]:
conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [61]:
cur.execute("""select * from orders""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141


In [62]:
cur.execute("""SELECT customerNumber, contactLastName, contactFirstName
                      FROM customers 
                      WHERE customerNumber in (SELECT customerNumber FROM orders 
                                               WHERE orderDate = '2003-01-31')""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [63]:
cur.execute("""SELECT productName, count(distinct orderNumber) as total_order,
                      sum(quantityOrdered) as quantity_sold
                      FROM products
                      JOIN orderdetails
                      USING (productCode)
                      GROUP BY productName
                      ORDER BY total_order desc;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,productName,total_order,quantity_sold
0,1992 Ferrari 360 Spider red,53,1808
1,18th Century Vintage Horse Carriage,28,907
2,1900s Vintage Bi-Plane,28,940
3,1900s Vintage Tri-Plane,28,1009
4,1913 Ford Model T Speedster,28,1038


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [64]:
cur.execute("""SELECT productName, count(distinct customerNumber) as total_num_ppl
                      FROM products 
                      JOIN orderdetails
                      USING(productCode)
                      JOIN orders
                      USING (orderNumber)
                      GROUP BY productName
                      ORDER BY total_num_ppl desc;
                      """)
                      
                      
                      
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,productName,total_num_ppl
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [71]:
#answer key
cur.execute("""SELECT distinct employeeNumber, officeCode, o.city, lastName, firstName
                      FROM employees
                      JOIN offices o
                      USING (officeCode)
                      JOIN customers
                      ON employees.employeeNumber = customers.SalesRepEmployeeNumber
                      JOIN orders 
                      USING (customerNumber)
                      JOIN orderdetails
                      USING (orderNumber)
                      WHERE productCode IN (SELECT productCode 
                                            FROM products
                                            JOIN orderdetails
                                            USING (productCode)
                                            JOIN orders
                                            USING (orderNumber)
                                            GROUP BY 1
                                            HAVING COUNT(distinct customerNumber) <= 20
                                            );""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,officeCode,city,lastName,firstName
0,1370,4,Paris,Hernandez,Gerard
1,1401,4,Paris,Castillo,Pamela
2,1621,5,Tokyo,Nishi,Mami
3,1501,7,London,Bott,Larry
4,1337,4,Paris,Bondur,Loui
5,1286,3,NYC,Tseng,Foon Yue
6,1166,1,San Francisco,Thompson,Leslie
7,1165,1,San Francisco,Jennings,Leslie
8,1612,6,Sydney,Marsh,Peter
9,1188,2,Boston,Firrelli,Julie


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [72]:
cur.execute("""SELECT * FROM customers""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300.00
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200.00
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,1504,81700.00


In [77]:
cur.execute("""SELECT employeeNumber, firstName, lastName, COUNT(distinct customerNumber)
                      FROM employees 
                      JOIN customers
                      ON employees.employeeNumber = customers.salesRepEmployeeNumber
                      GROUP BY 1
                                                      WHERE avg(creditlimit) > 15000 )""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

OperationalError: misuse of aggregate function avg()

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!